<a href="https://colab.research.google.com/github/yashika-git/Implementing_Segmentation/blob/main/Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U segmentation-models-pytorch
!pip -q install albumentations==0.4.6

In [ ]:
# Imports

import os 
import tqdm
import shutil
import random
from glob import glob

import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision.utils import save_image
import albumentations as A
from albumentations.pytorch.transforms import ToTensor
from PIL import Image, ImageOps
import segmentation_models_pytorch as smp


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
img_size = [512, 512]
root_dir = '/content/gdrive/MyDrive/task_2/'
image_dir = os.path.join(root_dir, 'images')
mask_dir = os.path.join(root_dir, 'masks')

In [ ]:
len(os.listdir(image_dir))

30

In [ ]:
len(os.listdir(mask_dir))

30

In [ ]:
mask_list = sorted(glob(f"{mask_dir}/*.png"))
image_list = sorted(glob(f"{image_dir}/*.png"))

In [ ]:
# Setting the seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

set_seed(42)

In [ ]:
# Train-validation split
train_image_list = image_list[:20]
train_mask_list = mask_list[:20]
valid_image_list = image_list[20:]
valid_mask_list = mask_list[20:]

In [ ]:
# Creating the Dataset Class

class Dataset(Dataset):
  def __init__(self, image_list, mask_list, transforms=None):
    self.image_list = image_list
    self.mask_list = mask_list
    self.transforms = transforms

  def __len__(self): 
    return len(self.image_list) 

  def __getitem__(self, index):
    image_path = image_list[index]
    mask_path = mask_list[index]
    image = np.array(ImageOps.grayscale(Image.open(image_path).convert('RGB')))
    mask = np.array(ImageOps.grayscale(Image.open(mask_path).convert('RGB')))
    mask = mask/255.0 

    if self.transforms:
           augmentation = self.transforms(image0=image, image=mask)
           image = augmentation["image0"]
           mask = augmentation["image"]  
           image = np.expand_dims(image,0)
           mask = np.expand_dims(mask,0)
          
    return image, mask

In [ ]:
transforms_train = A.Compose([
        A.Resize(256, 256),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),],
    additional_targets = {"image0":"image"}
)

transforms_valid = A.Compose([
        A.Resize(256, 256),
       ],
    additional_targets = {"image0":"image"}
)

In [ ]:
class Trainer:
    def __init__(
        self, 
        model, 
        device, 
        optimizer, 
        criterion
    ):
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.criterion = criterion

        self.best_valid_dice_loss = np.inf
        self.n_patience = 0
        self.lastmodel = None
        
    def fit(self, epochs, train_loader, valid_loader, patience):        
        for n_epoch in range(1, epochs + 1):
            self.info_message("EPOCH: {}", n_epoch)
            
            train_dice_loss = self.train_epoch(train_loader)
            valid_dice_loss = self.valid_epoch(valid_loader)
            
            self.info_message(
                "[Epoch Train: {}] dice_loss: {:.4f}",
                n_epoch, train_dice_loss
            )
            
            self.info_message(
                "[Epoch Valid: {}] dice_loss: {:.4f}",
                n_epoch, valid_dice_loss
            )

         
            if self.best_valid_dice_loss > valid_dice_loss: 
                self.save_model(n_epoch, valid_dice_loss)
                self.info_message(
                     "valid dice loss decreased from {:.4f} to {:.4f}. Saved model to '{}'", 
                    self.best_valid_dice_loss, valid_dice_loss, self.lastmodel
                )
                self.best_valid_dice_loss = valid_dice_loss
                self.n_patience = 0
            else:
                self.n_patience += 1
            
            if self.n_patience >= patience:
                self.info_message("\nValid dice loss didn't improve in last {} epochs.", patience)
                break
            
    def train_epoch(self, train_loader):
        self.model.train()
        sum_dice_loss = 0

        for step, batch in enumerate(train_loader, 1):
            image = batch[0].float()
            mask = batch[1].float()
            image = image.to(self.device)
            mask = mask.to(self.device)
            self.optimizer.zero_grad()
            outputs = self.model(image)
            
            loss = self.criterion(outputs, mask)
            loss.backward()

            sum_dice_loss += loss.detach().item()

            self.optimizer.step()
            
            message = 'Train Step {}/{}, train_dice_loss: {:.4f}'
            self.info_message(message, step, len(train_loader), sum_dice_loss/step, end="\r")
        
        return sum_dice_loss/len(train_loader)
    
    def valid_epoch(self, valid_loader):
        self.model.eval()
        sum_dice_loss = 0

        for step, batch in enumerate(valid_loader, 1):
            with torch.no_grad():
                image = batch[0].float().to(self.device)
                mask = batch[1].float().to(self.device)

                outputs = self.model(image)
                loss = self.criterion(outputs, mask)

                sum_dice_loss += loss.detach().item()
             
            message = 'Valid Step {}/{}, valid_dice_loss: {:.4f}'
            self.info_message(message, step, len(valid_loader), sum_dice_loss/step, end="\r")
        
        return sum_dice_loss/len(valid_loader)
    
    def save_model(self, n_epoch, loss):
        self.lastmodel = f"e{n_epoch}-dice_loss{loss:.3f}.pth"
        torch.save(
            {
                "model_state_dict": self.model.state_dict(),
                "optimizer_state_dict": self.optimizer.state_dict(),
                "best_valid_dice_loss": self.best_valid_dice_loss,
                "n_epoch": n_epoch,
            },
            self.lastmodel,
        )
    
    @staticmethod
    def info_message(message, *args, end="\n"):
        print(message.format(*args), end=end)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train():

    train_data_retriever = Dataset(
        train_image_list,
        train_mask_list,
        transforms_valid     
    )

    valid_data_retriever = Dataset(
        valid_image_list,
        valid_mask_list,
        transforms_valid       
    )

    train_loader = DataLoader(
        train_data_retriever,
        batch_size=4,
        shuffle=True,
        num_workers=2,
    )

    valid_loader = DataLoader(
        valid_data_retriever, 
        batch_size=4,
        shuffle=False,
        num_workers=2,
    )

    model = smp.Unet(
        encoder_name= 'efficientnet-b2',      
        encoder_weights="imagenet",     
        in_channels=1,                  
        classes=1,        
        activation="sigmoid",
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    criterion = smp.utils.losses.DiceLoss()

    trainer = Trainer(
        model, 
        device, 
        optimizer, 
        criterion
    )

    history = trainer.fit(
        50, 
        train_loader, 
        valid_loader,
        10,
    )
    return trainer.lastmodel

modelfiles = None

if not modelfiles:
  modelfiles = train()
print(modelfiles)

EPOCH: 1
[Epoch Train: 1] dice_loss: 0.5613
[Epoch Valid: 1] dice_loss: 0.8890
valid dice loss decreased from inf to 0.8890. Saved model to 'e1-dice_loss0.889.pth'
EPOCH: 2
[Epoch Train: 2] dice_loss: 0.4534
[Epoch Valid: 2] dice_loss: 0.9943
EPOCH: 3
[Epoch Train: 3] dice_loss: 0.4108
[Epoch Valid: 3] dice_loss: 0.9865
EPOCH: 4
[Epoch Train: 4] dice_loss: 0.3699
[Epoch Valid: 4] dice_loss: 0.8898
EPOCH: 5
[Epoch Train: 5] dice_loss: 0.3548
[Epoch Valid: 5] dice_loss: 0.8717
valid dice loss decreased from 0.8890 to 0.8717. Saved model to 'e5-dice_loss0.872.pth'
EPOCH: 6
[Epoch Train: 6] dice_loss: 0.3174
[Epoch Valid: 6] dice_loss: 0.8046
valid dice loss decreased from 0.8717 to 0.8046. Saved model to 'e6-dice_loss0.805.pth'
EPOCH: 7
[Epoch Train: 7] dice_loss: 0.3156
[Epoch Valid: 7] dice_loss: 0.6464
valid dice loss decreased from 0.8046 to 0.6464. Saved model to 'e7-dice_loss0.646.pth'
EPOCH: 8
[Epoch Train: 8] dice_loss: 0.3016
[Epoch Valid: 8] dice_loss: 0.6365
valid dice loss dec